In [1]:
from scipy import linalg 
import numpy as np
import math
import torch 
import torch.nn.functional as F

/home/sidd_s/scratch/miniconda3/envs/py39tor112/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# tr = np.load('/home/sidd_s/Diffusion_conditional_prior_segmentation/DDPS/tools/saved_files/matrix_numpy/transition_rate_matrix_city.npy')
# con = np.load('/home/sidd_s/Diffusion_conditional_prior_segmentation/DDPS/tools/saved_files/matrix_numpy/confusion_matrix_cityscapes_segformerb2.npy')
def betas_tt_arr(betas):
    betas_tt_arr = [np.sum(betas[:(t+1)]) for t in range(betas.shape[0])]
    return np.array(betas_tt_arr)

def simmat(betas_tt, transition_rate):
    matrix = linalg.expm(np.array(betas_tt * transition_rate, dtype=np.float64)) 
    return matrix

### different noise scheduling

In [3]:
# linear scheduler
betas = np.linspace(1e-6, 1e-4,20)
btt = betas_tt_arr(betas)

In [25]:
## cosine scheduler 

In [14]:
def cosine_fn(step):
      return np.cos((step / num_steps + s) / (1 + s) * np.pi / 2)
def schedule_fn(step):
      return np.clip(1 - (cosine_fn(step + 1) / cosine_fn(step)), 0, 0.999)

In [35]:
s = 1e-9
betas = []
num_steps = 20

In [36]:
betas = [schedule_fn(step) for step  in range(num_steps)]  
betas = np.array(betas)

In [222]:
# betas

In [134]:
def cos_alpha_schedule(time_step, N=20, att_1=0.99999, att_T=0.0001, ctt_1=0.0001, ctt_T=0.99999, exp=3):
    att = np.arange(0, time_step)
    att = (np.cos((att + time_step) * math.pi * 0.5 / time_step) + 1)**exp
    att = att * (att_1 - att_T) + att_T
    att = np.concatenate(([1], att))
    at = att[1:] / att[:-1]

    ctt = np.arange(0, time_step)
    ctt = (np.cos((ctt + time_step) * math.pi * 0.5 / time_step) + 1)**exp
    ctt = ctt * (ctt_1 - ctt_T) + ctt_T
    ctt = np.concatenate(([0], ctt))

    one_minus_ctt = 1 - ctt
    one_minus_ct = one_minus_ctt[1:] / one_minus_ctt[:-1]
    ct = 1 - one_minus_ct
    bt = (1 - at - ct) / N
    att = np.concatenate((att[1:], [1]))
    ctt = np.concatenate((ctt[1:], [0]))
    btt = (1 - att - ctt) / N
    # return at, bt, ct, att, btt, ctt
    return bt 

In [221]:
# betas = cos_alpha_schedule(20)
# betas 

In [225]:
# np.power(np.linspace(1e-4, 1e-1, num=20), math.e)

In [226]:
## testing log space vs linear space

In [229]:
betas = np.logspace(1, 2, base = 10, num=90)
betas

array([ 10.        ,  10.26209321,  10.53105571,  10.80706753,
        11.09031343,  11.38098302,  11.67927086,  11.98537662,
        12.2995052 ,  12.62186688,  12.95267744,  13.29215833,
        13.64053677,  13.99804598,  14.36492526,  14.7414202 ,
        15.12778282,  15.52427173,  15.93115236,  16.34869705,
        16.7771853 ,  17.21690393,  17.6681473 ,  18.13121744,
        18.60642434,  19.09408609,  19.59452913,  20.10808843,
        20.63510778,  21.17593994,  21.73094695,  22.30050032,
        22.8849813 ,  23.48478112,  24.10030129,  24.73195382,
        25.38016154,  26.04535835,  26.72798951,  27.42851197,
        28.14739464,  28.88511875,  29.6421781 ,  30.41907947,
        31.21634289,  32.03450204,  32.87410459,  33.73571256,
        34.61990268,  35.52726683,  36.45841237,  37.41396261,
        38.39455717,  39.40085245,  40.43352204,  41.4932572 ,
        42.5807673 ,  43.69678031,  44.84204325,  46.01732277,
        47.22340555,  48.46109895,  49.73123146,  51.03

In [3]:
betas = np.linspace(1e-4, 1e-2, 20)
betas

array([0.0001    , 0.00062105, 0.00114211, 0.00166316, 0.00218421,
       0.00270526, 0.00322632, 0.00374737, 0.00426842, 0.00478947,
       0.00531053, 0.00583158, 0.00635263, 0.00687368, 0.00739474,
       0.00791579, 0.00843684, 0.00895789, 0.00947895, 0.01      ])

In [231]:
# betas = np.logspace(-13, -9, base = math.e, num=20)
# betas

In [4]:
betas = np.logspace(-4, -2, base = 10, num=20)
betas

array([1.00000000e-04, 1.27427499e-04, 1.62377674e-04, 2.06913808e-04,
       2.63665090e-04, 3.35981829e-04, 4.28133240e-04, 5.45559478e-04,
       6.95192796e-04, 8.85866790e-04, 1.12883789e-03, 1.43844989e-03,
       1.83298071e-03, 2.33572147e-03, 2.97635144e-03, 3.79269019e-03,
       4.83293024e-03, 6.15848211e-03, 7.84759970e-03, 1.00000000e-02])

In [259]:
# betas[5]

In [220]:
# betas = np.geomspace(1e-6, 1e-4, 20)
# betas 
## similar to np.logspcace 

In [232]:
# betas = np.linspace(3.16e-6, 1e-4,20)
# betas

### Adjacency matrix calc from confusion matrix

In [5]:
def calculate_adjacency_matrix(confusion_matrix, k=3):
    ## calculate adjacency_matrix from confusion matrix 
    np.fill_diagonal(confusion_matrix,0) ## removing the dependency of the class with itself  ## its is an inplace argument 
    # indices = np.argpartition(confusion_matrix, -k, axis=1)[:, -k:]
    # top_k_values = np.take_along_axis(confusion_matrix, indices, axis=1)
    sorted_indices = np.argsort(confusion_matrix) 
    indices_as_ranks = np.argsort(np.argsort(confusion_matrix)) 
    indices_which_are_topk_as_bool = (indices_as_ranks >= confusion_matrix.shape[1] - k) # shape[1] used as we need to find it along row 
    adjacency_matrix = confusion_matrix * indices_which_are_topk_as_bool
    # print(adjacency_matrix)   
    ## for one hot adjacency matrix 
    adjacency_matrix_as_one_hot = adjacency_matrix.copy()
    adjacency_matrix_as_one_hot[adjacency_matrix_as_one_hot>0] = 1 
    
    return adjacency_matrix_as_one_hot

In [161]:
adj =calculate_adjacency_matrix(con, k=15)

In [162]:
adj.shape

(19, 19)

In [163]:
adj[0]

array([0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1.])

In [164]:
adj_s = (adj + adj.T) / 30

In [165]:
adj_s[0]

array([0.        , 0.06666667, 0.03333333, 0.06666667, 0.06666667,
       0.06666667, 0.        , 0.06666667, 0.06666667, 0.06666667,
       0.06666667, 0.06666667, 0.06666667, 0.06666667, 0.06666667,
       0.06666667, 0.03333333, 0.03333333, 0.06666667])

In [166]:
tr = adj_s - np.diagflat(np.sum(adj_s,axis =1))

In [167]:
np.diag(tr)

array([-1.03333333, -1.03333333, -1.1       , -1.06666667, -1.06666667,
       -1.1       , -0.7       , -1.1       , -1.1       , -0.93333333,
       -0.76666667, -1.1       , -1.03333333, -1.1       , -1.03333333,
       -1.        , -0.7       , -0.8       , -1.03333333])

In [168]:
tr.sum(0)

array([-3.19189120e-16, -3.05311332e-16, -2.08166817e-16, -1.94289029e-16,
       -1.94289029e-16, -3.05311332e-16, -2.77555756e-17, -3.19189120e-16,
       -2.08166817e-16, -1.24900090e-16, -1.38777878e-17, -1.94289029e-16,
       -1.94289029e-16, -1.94289029e-16, -1.94289029e-16, -8.32667268e-17,
       -9.71445147e-17, -9.71445147e-17, -2.22044605e-16])

In [169]:
tr.sum(1)

array([-9.71445147e-17, -8.32667268e-17, -9.71445147e-17,  2.77555756e-17,
        2.77555756e-17, -1.94289029e-16,  1.38777878e-17, -2.08166817e-16,
       -2.08166817e-16, -6.93889390e-17,  1.38777878e-17, -8.32667268e-17,
       -1.94289029e-16, -8.32667268e-17, -1.94289029e-16,  2.77555756e-17,
        1.38777878e-17,  1.38777878e-17,  0.00000000e+00])

In [170]:
tr[0]

array([-1.03333333,  0.06666667,  0.03333333,  0.06666667,  0.06666667,
        0.06666667,  0.        ,  0.06666667,  0.06666667,  0.06666667,
        0.06666667,  0.06666667,  0.06666667,  0.06666667,  0.06666667,
        0.06666667,  0.03333333,  0.03333333,  0.06666667])

### testing base matrix with various noise schedulers

In [185]:
m = simmat(8, tr)

In [186]:
m[0]

array([0.05287917, 0.05274396, 0.0526114 , 0.05268433, 0.05266573,
       0.0526417 , 0.05211753, 0.05264302, 0.05264789, 0.05272677,
       0.05256704, 0.05268148, 0.05270629, 0.05268148, 0.05263825,
       0.05262579, 0.05235408, 0.05265684, 0.05272727])

In [24]:
m[0]

array([9.99890007e-01, 1.99978501e-05, 5.99932838e-10, 1.99990333e-10,
       2.99979834e-10, 3.99965668e-10, 5.00033329e-11, 9.99910005e-06,
       7.49912006e-10, 1.99983001e-05, 5.00066658e-11, 9.99920004e-06,
       2.99979667e-10, 1.99978501e-05, 9.99930003e-06, 9.99925004e-06,
       1.49992500e-10, 1.99987334e-10, 9.99925004e-06])

In [262]:
np.diag(m)

array([0.05263675, 0.05263683, 0.0526356 , 0.05263658, 0.05263602,
       0.0526362 , 0.05264379, 0.05263768, 0.05263596, 0.05263697,
       0.05263654, 0.05263823, 0.05264545, 0.05263633, 0.05265286,
       0.05268067, 0.05445235, 0.05318652, 0.05264033])

In [263]:
m.sum(0)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1.])

In [264]:
m.sum(1)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1.])

### comparision with confusion matrix

In [4]:
con = np.load('/home/sidd_s/Diffusion_conditional_prior_segmentation/DDPS/tools/saved_files/matrix_numpy/confusion_matrix_cityscapes_segformerb2.npy')

In [5]:
np.diag(con)

array([3.42191258e+08, 4.62292430e+07, 1.94722792e+08, 4.86530400e+06,
       5.28670000e+06, 1.07098340e+07, 1.55824600e+06, 5.43912900e+06,
       1.53996056e+08, 5.48386100e+06, 3.01713920e+07, 1.09683500e+07,
       1.54797300e+06, 5.85725870e+07, 2.46212200e+06, 3.40205200e+06,
       8.91340000e+05, 6.00022000e+05, 5.86972000e+06])

In [25]:
np.diag(tr)

array([ -5628417.,  -7248092., -13844283.,  -2487401.,  -3157585.,
        -4894851.,   -523629.,  -1211845., -11490689.,  -3156210.,
        -1468612.,  -2064963.,   -811242.,  -2678828.,   -401651.,
         -322970.,   -191420.,   -234960.,  -1519438.])

In [23]:
con[0]

array([3.42191258e+08, 2.05861000e+06, 4.92220000e+04, 1.59470000e+04,
       2.46830000e+04, 3.69790000e+04, 1.00000000e+00, 1.24110000e+04,
       4.58740000e+04, 1.01316000e+05, 7.39700000e+03, 1.13438000e+05,
       7.62900000e+03, 4.67588000e+05, 8.48800000e+03, 2.04460000e+04,
       5.30000000e+03, 4.17300000e+03, 5.12500000e+04])

In [24]:
con[2]

array([1.14970000e+04, 2.52640000e+05, 1.94722792e+08, 1.44523000e+05,
       1.85004000e+05, 7.84863000e+05, 9.77150000e+04, 2.29575000e+05,
       3.30994100e+06, 1.56480000e+04, 4.05660000e+05, 2.95075000e+05,
       2.96470000e+04, 1.78111000e+05, 2.47580000e+04, 1.55890000e+04,
       1.21720000e+04, 6.23000000e+03, 1.73833000e+05])

### testing rough work

In [8]:
con = torch.tensor(np.load('/home/sidd_s/Diffusion_conditional_prior_segmentation/DDPS/tools/saved_files/matrix_numpy/confusion_matrix_cityscapes_segformerb2.npy'))

In [12]:
test = torch.ones((19,19))

In [14]:
test.shape

torch.Size([19, 19])

In [17]:
test_p = F.pad(input=test, pad=(0, 1, 0, 1), mode='constant', value=0)

In [20]:
test_p[:,-1]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [21]:
test_p[-1,:]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [22]:
test_p[0]

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 0.])

In [23]:
t2 = torch.matmul(test_p, test_p)

In [24]:
t2.shape

torch.Size([20, 20])

In [25]:
t2[-1,:] 

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [26]:
t2[:, -1]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [27]:
t2[0]

tensor([19., 19., 19., 19., 19., 19., 19., 19., 19., 19., 19., 19., 19., 19.,
        19., 19., 19., 19., 19.,  0.])

In [30]:
t3 = torch.matmul(t2, test_p)

In [32]:
# t3

In [39]:
test = torch.ones((20,20,20))

In [40]:
test = test.tril()

In [41]:
test.transpose()

tensor([[[1., 0., 0.,  ..., 0., 0., 0.],
         [1., 1., 0.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         ...,
         [1., 1., 1.,  ..., 1., 0., 0.],
         [1., 1., 1.,  ..., 1., 1., 0.],
         [1., 1., 1.,  ..., 1., 1., 1.]],

        [[1., 0., 0.,  ..., 0., 0., 0.],
         [1., 1., 0.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         ...,
         [1., 1., 1.,  ..., 1., 0., 0.],
         [1., 1., 1.,  ..., 1., 1., 0.],
         [1., 1., 1.,  ..., 1., 1., 1.]],

        [[1., 0., 0.,  ..., 0., 0., 0.],
         [1., 1., 0.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         ...,
         [1., 1., 1.,  ..., 1., 0., 0.],
         [1., 1., 1.,  ..., 1., 1., 0.],
         [1., 1., 1.,  ..., 1., 1., 1.]],

        ...,

        [[1., 0., 0.,  ..., 0., 0., 0.],
         [1., 1., 0.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         ...,
         [1., 1., 1.,  ..., 1., 0., 0.],
         [1., 1., 1., 

### testing for which Transition matrix type to be used 

In [7]:
def calculate_adjacency_matrix(confusion_matrix, k=3):
    ## calculate adjacency_matrix from confusion matrix 
    np.fill_diagonal(confusion_matrix,0) ## removing the dependency of the class with itself  ## its is an inplace argument 
    sorted_indices = np.argsort(confusion_matrix) 
    indices_as_ranks = np.argsort(np.argsort(confusion_matrix)) 
    indices_which_are_topk_as_bool = (indices_as_ranks >= confusion_matrix.shape[1] - k) # shape[1] used as we need to find it along row 
    adjacency_matrix = confusion_matrix * indices_which_are_topk_as_bool
    adjacency_matrix_as_one_hot = adjacency_matrix.copy()
    adjacency_matrix_as_one_hot[adjacency_matrix_as_one_hot>0] = 1 
    
    return adjacency_matrix_as_one_hot

In [8]:
adj =calculate_adjacency_matrix(con, k=5)

In [9]:
adj.shape

(19, 19)

In [10]:
adj[0]

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0.,
       0., 1.])

In [14]:
adjacency_matrix_soft = (adj + adj.T) / 10

In [16]:
transition_rate = adjacency_matrix_soft - np.diagflat(np.sum(adjacency_matrix_soft, axis=1))

In [18]:
np.diag(transition_rate)

array([-1.1, -1.3, -2.1, -0.7, -0.9, -1.4, -0.7, -0.7, -2.1, -0.8, -0.9,
       -1.1, -0.8, -1.3, -0.5, -0.6, -0.5, -0.5, -1. ])

In [19]:
transition_rate.sum(0)

array([-8.32667268e-17, -1.94289029e-16, -2.77555756e-17,  2.77555756e-17,
       -1.38777878e-16, -8.32667268e-17, -2.77555756e-17, -5.55111512e-17,
       -2.49800181e-16,  0.00000000e+00,  2.77555756e-17, -2.77555756e-17,
       -2.77555756e-17, -1.38777878e-16,  0.00000000e+00,  2.77555756e-17,
        0.00000000e+00,  2.77555756e-17, -1.11022302e-16])

In [21]:
transition_rate.sum(1)

array([-8.32667268e-17,  0.00000000e+00, -1.38777878e-16,  0.00000000e+00,
        0.00000000e+00,  1.11022302e-16, -5.55111512e-17, -5.55111512e-17,
       -5.55111512e-17, -5.55111512e-17,  8.32667268e-17, -2.77555756e-17,
       -2.77555756e-17,  8.32667268e-17,  5.55111512e-17,  8.32667268e-17,
        0.00000000e+00,  2.77555756e-17,  0.00000000e+00])

In [50]:
betas = 2.5e1

In [51]:
matrix = linalg.expm(
                        np.array(betas * transition_rate, dtype=np.float64))

In [52]:
# matrix.shape

In [53]:
matrix[0]

array([0.05263213, 0.05263174, 0.05263146, 0.05263165, 0.05263137,
       0.05263105, 0.05262969, 0.05263036, 0.05263136, 0.05263199,
       0.05262998, 0.05263259, 0.05263293, 0.05263214, 0.05263243,
       0.05263111, 0.0526297 , 0.05263354, 0.05263276])

In [54]:
## next confusion matrix of results in darkzurich using cityscapes 

In [55]:
matrix[1]

array([0.05263174, 0.05263489, 0.05263162, 0.05263964, 0.05263847,
       0.05263582, 0.05263814, 0.05263793, 0.05263348, 0.05263725,
       0.05263618, 0.0526242 , 0.05262041, 0.05262689, 0.05263254,
       0.05263293, 0.0526318 , 0.05261364, 0.05262243])

### checking betas for similarity matrix derived from protos

In [1]:
import torch 
import seaborn as sns
import torch.nn.functional as F

/home/sidd_s/scratch/miniconda3/envs/py39tor112/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model = torch.load('/home/sidd_s/scratch/saved_models/Protoseg/hrnet_w48_proto_lr1x_hrnet_proto_80k_latest.pth')
protos = model['state_dict']['module.prototypes']

In [6]:
def similarity_among_classes(protos):
    cos = torch.nn.CosineSimilarity(dim=0)
    similarity_matrix = []
    for i in range(19):
        per_row_similarity = [] 
        for j in range(19):
            per_row_similarity.append(cos(protos[i][0], protos[j][0]))
        similarity_matrix.append(per_row_similarity)
    similarity_matrix_tensor = torch.FloatTensor(similarity_matrix)     

    sim_test = similarity_matrix_tensor.clone() 
    sim_test.fill_diagonal_(-1e17) ## filling extremely low numbers at diagonals for removing them from similarity consideration
    probas_sim_test = F.softmax(sim_test, dim=1) ## applying softmax to convert into probability distribution 
    
    return probas_sim_test, similarity_matrix_tensor

In [38]:
probas_sim_test, similarity_matrix_tensor = similarity_among_classes(protos)

In [39]:
probas_sim_test[0]

tensor([0.0000, 0.0703, 0.0552, 0.0538, 0.0541, 0.0544, 0.0530, 0.0538, 0.0544,
        0.0589, 0.0532, 0.0551, 0.0540, 0.0570, 0.0543, 0.0543, 0.0549, 0.0543,
        0.0550])

In [40]:
betas.shape

(20,)

In [42]:
probas_sim_test.sum(1)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000])

In [58]:
similarity_matrix = probas_sim_test.clone() 
similarity_matrix = similarity_matrix + similarity_matrix.T ## symmetricity required in transition rate 

In [59]:
similarity_matrix.shape

torch.Size([19, 19])

In [60]:
similarity_matrix[1]

tensor([0.1393, 0.0000, 0.1061, 0.1139, 0.1095, 0.1109, 0.1049, 0.1060, 0.1046,
        0.1433, 0.1040, 0.1093, 0.1055, 0.1082, 0.1068, 0.1069, 0.1067, 0.1078,
        0.1125])

In [61]:
similarity_matrix = np.array(similarity_matrix)

In [62]:
transition_rate = similarity_matrix - np.diagflat(np.sum(similarity_matrix, axis=1)) ## transition rate matrix 

In [63]:
transition_rate.sum(1)

array([ 6.7055225e-08, -8.1956387e-08,  1.6391277e-07, -7.4505806e-08,
        5.2154064e-08,  8.9406967e-08,  6.7055225e-08, -1.1920929e-07,
        1.1920929e-07,  1.2665987e-07, -8.9406967e-08,  1.2665987e-07,
        0.0000000e+00, -8.9406967e-08,  8.9406967e-08, -7.4505806e-09,
        7.4505806e-08,  0.0000000e+00,  0.0000000e+00], dtype=float32)

In [64]:
transition_rate.sum(0)

array([-5.2154064e-08, -2.6077032e-07, -7.4505806e-08, -3.1292439e-07,
        5.2154064e-08,  2.9802322e-08,  1.2665987e-07, -5.9604645e-08,
        5.9604645e-08, -2.2351742e-08, -1.4901161e-07, -2.2351742e-08,
        2.3841858e-07, -1.1920929e-07,  1.4901161e-07, -7.4505806e-09,
        1.9371510e-07,  0.0000000e+00,  0.0000000e+00], dtype=float32)

In [65]:
betas = torch.tensor(betas)

/tmp/ipykernel_37217/3861057766.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  betas = torch.tensor(betas)


In [71]:
t = 18
betas_tt = torch.sum(betas[:t+1]).item() ## since t is starting from 0 ## cummulative steps matrix expo calc

In [72]:
betas_tt

0.00091

In [84]:
betas_tt = 1e2

In [85]:
matrix = linalg.expm(np.array(betas_tt * transition_rate, dtype=np.float64)) 

In [86]:
matrix.shape

(19, 19)

In [87]:
matrix[0]

array([0.05263165, 0.05263165, 0.05263165, 0.05263165, 0.05263165,
       0.05263165, 0.05263165, 0.05263165, 0.05263165, 0.05263165,
       0.05263165, 0.05263165, 0.05263166, 0.05263165, 0.05263165,
       0.05263165, 0.05263166, 0.05263165, 0.05263165])

In [76]:
transition_rate[0]

array([-1.985805  ,  0.1393426 ,  0.10665143,  0.10675085,  0.10680521,
        0.10621709,  0.10629451,  0.10738534,  0.10527728,  0.11741515,
        0.10804304,  0.10950789,  0.10707401,  0.11328368,  0.10873927,
        0.10853966,  0.11065185,  0.10873567,  0.10909052], dtype=float32)

In [88]:
# using sinkhorn algorithm

In [97]:
beta_t = 1e-1

In [98]:
matrix = probas_sim_test.clone() 
matrix = beta_t * matrix
# ## sinkhorn algo for base matrix
for _ in range(5): # number of iterations is a hyperparameter of sinkhorn's algo ## till in covergence 
    matrix = matrix / matrix.sum(1, keepdims=True)
    matrix = matrix / matrix.sum(0, keepdims=True)

In [99]:
matrix.shape

torch.Size([19, 19])

In [100]:
matrix[0]

tensor([0.0000, 0.0699, 0.0520, 0.0538, 0.0535, 0.0525, 0.0541, 0.0543, 0.0515,
        0.0594, 0.0558, 0.0552, 0.0538, 0.0571, 0.0553, 0.0551, 0.0566, 0.0553,
        0.0548])

In [101]:
matrix.sum(0)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000])

In [102]:
matrix.sum(1)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000])

In [103]:
matrix[1]

tensor([0.0699, 0.0000, 0.0512, 0.0567, 0.0542, 0.0542, 0.0528, 0.0531, 0.0506,
        0.0717, 0.0531, 0.0545, 0.0525, 0.0539, 0.0537, 0.0536, 0.0540, 0.0542,
        0.0559])

In [ ]:
## now, cumulative using 